# VactionPy


This file contains the following:
    - A heat map that displays the humidity for every city from data collected from OpenWeatherMap.org (see WeatherPy)
    - Data for cafes, bulgogi restaurants, and museums in Seoul, Korea pulled from Google Places API
    - Map of the locations as pins with details

In [1]:
# Dependencies and Setup

import pandas as pd
import numpy as np
import requests
import gmaps
import json
from pprint import pprint

# Import API key for google

from api_keys import g_key

# Read the weather data

cities_weather_data = pd.read_csv("../WeatherPy/output_data/clean_cities_weather_data.csv", delimiter=",",low_memory=False)

# Display the first few rows of weather data

cities_weather_data.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ushuaia,40,AR,1569303867,81,-54.81,-68.31,44.60,5.82
1,kapaa,90,US,1569303701,83,22.08,-159.32,80.60,6.93
2,kavaratti,75,IN,1569303892,79,10.57,72.64,84.20,4.70
3,rikitea,74,PF,1569303892,72,-23.12,-134.97,69.89,12.80
4,weinan,0,CN,1569303893,60,34.50,109.46,78.01,2.24


In [2]:
# Configure gmaps

gmaps.configure(api_key=g_key)

# Set the latitude/longitude and humidity data to variables
# Set the data as type float

locations = cities_weather_data[["Lat", "Lng"]].astype(float)
humidity = cities_weather_data["Humidity"].astype(float)

# Set the gmaps figure

fig_heat_map = gmaps.figure()

# Create a heat map using gmaps
# Locations will be the lat and long of the above data
# Weights will be the humidity where the max intensity will be set to the highest value (100%)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Add the heat layer to the map

fig_heat_map.add_layer(heat_layer)

# Display the map

fig_heat_map

Figure(layout=FigureLayout(height='420px'))

In [3]:
# Rename columns to be able to use query() 

cities_weather_data = cities_weather_data.rename(columns={"Max Temp" : "Max_Temp", "Wind Speed":"Wind_Speed"})

# Display updated data frame

cities_weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,ushuaia,40,AR,1569303867,81,-54.81,-68.31,44.60,5.82
1,kapaa,90,US,1569303701,83,22.08,-159.32,80.60,6.93
2,kavaratti,75,IN,1569303892,79,10.57,72.64,84.20,4.70
3,rikitea,74,PF,1569303892,72,-23.12,-134.97,69.89,12.80
4,weinan,0,CN,1569303893,60,34.50,109.46,78.01,2.24


In [4]:
# Using query(), narrow down the data to find the city with my ideal weather conditions

ideal_weather = cities_weather_data.query('40<Humidity<60 & 0<Cloudiness<20 & 65<Max_Temp<85 & 0<Wind_Speed<5')

# Display the data

ideal_weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
147,seoul,1,KR,1569303725,44,37.57,126.98,78.8,4.7


In [ ]:
# Set base url

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters for cafes

cafe_params = {
    "location": "37.57, 126.9",
    "keyword": "coffee",
    "radius":50000,
    "type": "cafe",
    "key": g_key
    }

# Request information from Google Places API

cafe_response = requests.get(base_url, params=cafe_params)

# Set to json() format

cafe_places_data = cafe_response.json()

# Pretty print the data
# Removed output since I will be pushing this to github

pprint(cafe_places_data)

In [6]:
# Create empty data frame to store cafe data later

cafes_df=pd.DataFrame()
cafes_df["Cafe_Name"]=""
cafes_df["Lat"]=""
cafes_df["Lng"]=""
cafes_df["Address"]=""
cafes_df["Rating"]=""

# Display empty data frame

cafes_df.head()

,Cafe_Name,Lat,Lng,Address,Rating


In [7]:
# Create for loop to gather data from the API requests and put the data into the empty cafe data frame

for i in np.arange(0, len(cafe_places_data),1):
    cafes_df.loc[i, "Cafe_Name"] = cafe_places_data["results"][i]["name"]
    cafes_df.loc[i, "Lat"] = cafe_places_data["results"][i]["geometry"]["location"]["lat"]
    cafes_df.loc[i, "Lng"] = cafe_places_data["results"][i]["geometry"]["location"]["lng"]
    cafes_df.loc[i, "Address"] = cafe_places_data["results"][i]["vicinity"]
    cafes_df.loc[i, "Rating"] = cafe_places_data["results"][i]["rating"]
    
# Display the cafe data frame

cafes_df

,Cafe_Name,Lat,Lng,Address,Rating
0,EL CAFE coffee roasters,37.5367,126.892,"7 Yangpyeong-ro 21na-gil, Yangpyeongdong 5(o)-...",4.4
1,Cafe de Paris,37.562,126.986,"40 Myeongdong 10-gil, Chungmuro 2(i)-ga, Jung-gu",3.9
2,Cafe Colon,37.5233,126.925,"25-11 Yeoui-dong, Yeongdeungpo-gu",4.1
3,Cafe De Paris,37.5221,127.024,"59 Nonhyeon-ro 161-gil, Sinsa-dong, Gangnam-gu",4.2


In [8]:
# Create info box for pins

info_box_template =info_box_template = """
<dl>
<dt>Name</dt><dd>{Cafe_Name}</dd>
<dt>Address</dt><dd>{Address}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
</dl>
"""
# Use list comprehension to get cafe data into the info box

cafe_info = [info_box_template.format(**row) for index, row in cafes_df.iterrows()]

# Set the locations to the lat and long from the cafe data frame

cafe_locations = cafes_df[["Lat", "Lng"]]

# Create a marker layer for the cafe locations and data

marker_layer = gmaps.marker_layer(cafe_locations, info_box_content=cafe_info)

# Add the layer to the humidity heat map

fig_heat_map.add_layer(marker_layer)

# Display the pins of the cafes on the heat map

fig_heat_map

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Set parameters for bulgogi restuarants

params_restaurants = {
    "location": "37.57, 126.9",
    "keyword": "bulgogi",
    "radius":5000,
    "type": "restaurant",
    "key": g_key
    }

# Request information from Google Places API

response_restaurants = requests.get(base_url, params=params_restaurants)

# Set to json() format

places_data_restuarants = response_restaurants.json()

# Pretty print the data
# Removed output since I will be pushing this to github

pprint(places_data_restuarants)

In [10]:
# Create empty data frame for restaurants

restaurants=pd.DataFrame()
restaurants["Restaurant_Name"]=""
restaurants["Lat"]=""
restaurants["Lng"]=""
restaurants["Address"]=""
restaurants["Rating"]=""

# Display empty data frame

restaurants.head()

,Restaurant_Name,Lat,Lng,Address,Rating


In [11]:
# Create for loop to gather data from the API requests and put the data into the empty restaurant data frame

for i in np.arange(0, len(places_data_restuarants),1):
    restaurants.loc[i, "Restaurant_Name"] = places_data_restuarants["results"][i]["name"]
    restaurants.loc[i, "Lat"] = places_data_restuarants["results"][i]["geometry"]["location"]["lat"]
    restaurants.loc[i, "Lng"] = places_data_restuarants["results"][i]["geometry"]["location"]["lng"]
    restaurants.loc[i, "Address"] = places_data_restuarants["results"][i]["vicinity"]
    restaurants.loc[i, "Rating"] = places_data_restuarants["results"][i]["rating"]

# Display the data frame

restaurants

,Restaurant_Name,Lat,Lng,Address,Rating
0,Sandeomi Bulgogi,37.5494,126.921,"47 Eoulmadang-ro, Seogyo-dong, Mapo-gu",3.8
1,MASTER BULGOGI,37.5486,126.922,"25 Wausan-ro 11-gil, Sangsu-dong, Mapo-gu",0
2,Yetmat Seoul Bulgogi,37.5495,126.932,"71 Seogang-ro, Changjeon-dong, Mapo-gu",3.8


In [ ]:
# Set parameters for art museums

params_museums = {
    "location": "37.57, 126.9",
    "keyword": "art",
    "radius":5000,
    "type": "museum",
    "key": g_key
    }

# Request information from Google Places API

response_museums = requests.get(base_url, params=params_museums)

# Set to json() format

places_data_museums = response_museums.json()

# Pretty print the data
# Removed output since I will be pushing this to github

pprint(places_data_museums)

In [13]:
# Create empty museum data frame

museums=pd.DataFrame()
museums["Museum_Name"]=""
museums["Lat"]=""
museums["Lng"]=""
museums["Address"]=""
museums["Rating"]=""

# Display empty data frame

museums.head()

,Museum_Name,Lat,Lng,Address,Rating


In [14]:
# Create for loop to gather data from the API requests and put the data into the empty museum data frame

for i in np.arange(0, len(places_data_museums),1):
    museums.loc[i, "Museum_Name"] = places_data_museums["results"][i]["name"]
    museums.loc[i, "Lat"] = places_data_museums["results"][i]["geometry"]["location"]["lat"]
    museums.loc[i, "Lng"] = places_data_museums["results"][i]["geometry"]["location"]["lng"]
    museums.loc[i, "Address"] = places_data_museums["results"][i]["vicinity"]
    museums.loc[i, "Rating"] = places_data_museums["results"][i]["rating"]

# Display museum data frame

museums

,Museum_Name,Lat,Lng,Address,Rating
0,Trickeye Museum Seoul,37.5536,126.922,"Mapo-gu, Seogyo-dong, 홍익로 3길 20",4.2
1,Gimdaljin Art materials Museum,37.6,126.957,"Jongno-gu, 홍지동 44",4.2
2,Hongik Museum of Art,37.5507,126.926,"Mapo-gu, Wausan-ro, 94 문헌관 4층",0


In [17]:
# Create new map for cafe, restaurant, and museum data

fig_all_pins = gmaps.figure()

# Set info box for cafe, restaurant, museum data

cafe_info_box_template =info_box_template = """
<dl>
<dt>Name</dt><dd>{Cafe_Name}</dd>
<dt>Address</dt><dd>{Address}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
</dl>
"""
rest_info_box_template =info_box_template = """
<dl>
<dt>Name</dt><dd>{Restaurant_Name}</dd>
<dt>Address</dt><dd>{Address}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
</dl>
"""
muse_info_box_template =info_box_template = """
<dl>
<dt>Name</dt><dd>{Museum_Name}</dd>
<dt>Address</dt><dd>{Address}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
</dl>
"""

# Use list comprehension to get cafe, restaurnt, and museum data into the info box

cafe_info = [cafe_info_box_template.format(**row) for index, row in cafes_df.iterrows()]
rest_info = [rest_info_box_template.format(**row) for index, row in restaurants.iterrows()]
muse_info = [muse_info_box_template.format(**row) for index, row in museums.iterrows()]

# Set the locations to the lat and long from the cafe, restaurant, and museum data frame

cafe_locations = cafes_df[["Lat", "Lng"]]
rest_locations = restaurants[["Lat", "Lng"]]
muse_locations = museums[["Lat", "Lng"]]

# Create pins for cafe, restaurant, and museum data

cafe_layer = gmaps.symbol_layer(cafe_locations, info_box_content=cafe_info, fill_color="red", scale = 6, stroke_color="black", stroke_opacity=0.00009)
rest_layer= gmaps.symbol_layer(rest_locations, info_box_content=rest_info, fill_color="blue", scale = 6, stroke_color="black", stroke_opacity=0.00009)
muse_layer= gmaps.symbol_layer(muse_locations, info_box_content=muse_info, fill_color="green", scale = 6, stroke_color="black", stroke_opacity=0.00009)

# Add the cafe, restaurant, and museum layers to map

fig_all_pins.add_layer(cafe_layer)
fig_all_pins.add_layer(rest_layer)
fig_all_pins.add_layer(muse_layer)

# Display the map

fig_all_pins

Figure(layout=FigureLayout(height='420px'))